In [6]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [54]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0x364A_23F2)
N = 500
n = 10
X = np.random.uniform(low=-1, high=1, size=(N, n))
y = np.zeros(N)

_c = np.random.randn(n)
_d = np.random.randn(n)

#We generate data as gaussians with mean and variance dependent on the feature x.
for i in range(N):
    y[i] = np.random.normal(loc=_c.T @ X[i], scale=np.abs(_d.T @ X[i]))

In [76]:
import cvxpy as cp
import numpy as np

def solve_decile_regression(include_constraints = True):
  v = cp.Variable((9, 1))
  theta = cp.Variable((9, n))
  q = v + theta @ X.T

  constraints = []

  temp = np.zeros((9, N))

  obj = cp.sum([0.5 * cp.abs(q[j, i] - y[i] ) + (0.5 - 0.1*(j + 1)) * (q[j, i] - y[i] ) for j in np.arange(9) for i in np.arange(N)])

  if include_constraints:
    for i in np.arange(N):
      for j in range(8):
        constraints += [q[j+1][i] >= q[j][i]]
      
  prob = cp.Problem(cp.Minimize(cp.sum(obj)), constraints)
  prob.solve()

  v_opt = v.value
  theta_opt = theta.value

  return v_opt, theta_opt

In [77]:
v_opt, theta_opt = solve_decile_regression()

In [81]:
v_opt

array([[-2.56452864],
       [-1.34672048],
       [-0.73899949],
       [-0.3008427 ],
       [-0.02471041],
       [ 0.26075639],
       [ 0.86121569],
       [ 1.46420208],
       [ 2.71852354]])

In [82]:
theta_opt[0, :]

array([ 0.42789578, -1.73897692, -1.29932412,  0.17789533,  1.3585415 ,
       -0.20079958, -0.10278428,  1.54194391,  0.23492133,  1.17322729])

In [78]:
v_opt_uncons, theta_opt_uncons = solve_decile_regression(include_constraints = False)

In [83]:
v_opt_uncons

array([[-2.54413669],
       [-1.34728588],
       [-0.74604442],
       [-0.27686327],
       [-0.03760847],
       [ 0.25808255],
       [ 0.88644751],
       [ 1.48845713],
       [ 2.71852354]])

In [85]:
theta_opt_uncons[0, :]

array([ 0.53330889, -1.77468807, -1.31942949,  0.1528987 ,  1.44823285,
       -0.13765922, -0.17844532,  1.54278969,  0.09559465,  1.06927663])

In [100]:
def validate_decile(v_opt, theta_opt):
  count_tracker = np.zeros(9)
  for i in np.arange(N):
    q_hat = v_opt + theta_opt @ X[i, :].reshape(-1, 1)
    
    for j in np.arange(9):
      if y[i] <= q_hat[j]:
        count_tracker[j] += 1

  return count_tracker / N
  
validate_decile(v_opt, theta_opt)

array([0.098, 0.196, 0.3  , 0.4  , 0.508, 0.6  , 0.698, 0.804, 0.902])

In [101]:
validate_decile(v_opt_uncons, theta_opt_uncons)

array([0.1  , 0.2  , 0.298, 0.4  , 0.496, 0.6  , 0.696, 0.798, 0.902])

In [124]:
def find_feature_vector(v_opt, theta_opt):
  count_tracker = np.zeros(9)
  for i in np.arange(N):
    q_hat = v_opt + theta_opt @ X[i, :].reshape(-1, 1)
    
    if not np.all(D @ q_hat >= 0):
      print(f"Inconsistent feature vector: \n{X[i, :]}")
      print(f"D @ q_hat: \n{D @ q_hat}")
      break

find_feature_vector(v_opt_uncons, theta_opt_uncons)

Inconsistent feature vector: 
[-0.24411579  0.02871262 -0.97031735  0.2588894   0.73881531  0.12001584
 -0.65677871  0.74241652  0.22339428 -0.55781893]
D @ q_hat: 
[[ 0.01324647]
 [ 0.41874202]
 [-0.0072486 ]
 [ 0.20403505]
 [ 0.24390981]
 [ 0.97426729]
 [ 0.52389424]
 [ 0.9822208 ]]


In [119]:
q_hat = v_opt + theta_opt @ X[i, :].reshape(-1, 1)
if not np.all(D @ q_hat >= 0)

True

In [70]:
  v = cp.Variable((9, 1))
  theta = cp.Variable((9, n))
  q = v + theta @ X.T

  constraints = []

  temp = np.zeros((9, N))

  obj = cp.sum([0.5 * cp.abs(q[j, i] - y[i] ) + (0.5 - 0.1*j) * (q[j, i] - y[i] ) for j in np.arange(9) for i in np.arange(N)])

  prob = cp.Problem(cp.Minimize(cp.sum(temp)), constraints)
  prob.solve()

  v_opt = v.value
  theta_opt = theta.value

  return v_opt, theta_opt

NameError: ignored

b)

In [ ]:
### c)

In [107]:
def solve_unconstrained():

  TH = np.zeros((9, n))
  V = np.zeros(9)

  for j in np.arange(1, 10):
  
    theta = cp.Variable(n)
    v = cp.Variable()

    q_hat = X @ theta + np.ones(N)*v
    u = q_hat - y
    phi = 0.5 * cp.abs(u) + (0.5 - 0.1*j) * u
  
    objective = cp.sum(phi)
    constraints = []

    cp.Problem(cp.Minimize(objective), constraints).solve()
    TH[j-1, :] = theta.value
    V[j-1] = v.value

  return TH, V

TH, V = solve_unconstrained()

In [108]:
V

array([-2.54413669, -1.34728587, -0.74604469, -0.2768633 , -0.03760847,
        0.25808255,  0.88644751,  1.4884572 ,  2.71852354])

In [109]:
TH[0, :]

array([ 0.53330888, -1.77468806, -1.3194295 ,  0.1528987 ,  1.44823285,
       -0.13765923, -0.17844531,  1.54278969,  0.09559465,  1.06927663])

In [110]:
validate_decile(V.reshape(-1, 1), TH)

array([0.098, 0.2  , 0.296, 0.402, 0.502, 0.6  , 0.698, 0.8  , 0.9  ])